# Таксация

#### Структура датафрейма топографического плана

|  origin_number   |      number_position       |   split_number    |                  type                  |                geometry                |       size        |
|:----------------:|:--------------------------:|:-----------------:|:--------------------------------------:|:--------------------------------------:|:-----------------:|
|       str        |       shapely Point        |        str        |                  str                   |  shapely Point / LineString / Polygon  |   None / float    |
|  исходный номер  |  позиция исходного номера  | разделенный номер |  тип геометрии объекта растительности  |    геометрия объекта растительности    |  длина / площадь  |

Типы объектов растительности:
* Дерево `shapely Point`
* Полоса деревьев `shapely LineString`
* Контур растительности `shapely Polygon`

#### Структура датафрейма ведомости таксации

|  origin_number   |     name     |    split_number    |  quantity  | height | diameter |  quality   |
|:----------------:|:------------:|:------------------:|:----------:|:------:|:--------:|:----------:|
|       str        |     str      |        str         |    str     |  str   |   str    |    str     |
|  исходный номер  | наименование | разделенный номер  | количество | высота | диаметр  | состояние  |

TODO:
* разбить датафрейм ведомости таксации по разделенным номерам
* проверить наличие всех разделенных номеров в обоих датафреймах через разницу множеств по колонкам split_number
* добавить в датафрейм ведомости таксации колонку type и size
* сравнить колонки type и size в обоих датафреймах с использованием сортировки колонок split_number
* создать общий датафрейм **| origin_number | split_number | number_position | name | quantity | height | diameter | quality |**
* создать функцию для расчета компенсационных посадок/выплат для применения к строке общего датафрейма и создания доп колонки со строкой расчета


In [1]:
import numpy as np
# %reload_ext autoreload
# %autoreload all
from IPython.display import display

from pathlib import Path

import pandas as pd

from src.processing.topographic_plan import create_topographic_plan
from src.processing.taxation_list import create_taxation_list

## Обработка топографического плана

Чтение dxf файла топографического чертежа и создание объекта топографического плана.

In [2]:
topographic_plan = create_topographic_plan(dxf_path=Path("data/example_1_topographic_plan.dxf"),
                                           numbers_layers=["номера"],
                                           lines_layers=["полосы"],
                                           contours_layers=["контуры"],
                                           max_distance=0.01)
topographic_plan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   origin_number    36 non-null     object 
 1   number_position  36 non-null     object 
 2   split_number     36 non-null     object 
 3   type             36 non-null     object 
 4   geometry         36 non-null     object 
 5   size             10 non-null     float64
dtypes: float64(1), object(5)
memory usage: 1.8+ KB


### Проверка корректности датафрейма топографического плана

In [3]:
def test_unsplit_number() -> None:
    """
    Проверка на наличие неразделенных номеров
    :return: None
    """
    if topographic_plan["split_number"].isna().sum() > 0:
        display(topographic_plan[topographic_plan["split_number"].isna()])
        raise ValueError(f"Обнаружены неразделенные номера.")

test_unsplit_number()

In [4]:
def test_duplicated_slit_number() -> None:
    """
    Проверка на наличие дубликатов разделенных номеров
    :return: None
    """
    duplicated_slit_numbers = topographic_plan.duplicated(subset=['split_number'], keep=False)
    if any(duplicated_slit_numbers):
        display(topographic_plan[duplicated_slit_numbers])
        raise ValueError(f"Обнаружены дубликаты разделенных номеров.")

test_duplicated_slit_number()

,origin_number,number_position,split_number,type,geometry,size
33,28,POINT (3097.319681942773 1694.253334704742),28,Polygon,"POLYGON ((3100.419036451997 1697.392114117334,...",13.404293
34,28,POINT (3098.977617627182 1697.65937651281),28,Polygon,"POLYGON ((3100.419036451997 1697.392114117334,...",13.404293
35,28,POINT (3100.419036451997 1697.392114117334),28,Polygon,"POLYGON ((3100.419036451997 1697.392114117334,...",13.404293


ValueError: Обнаружены дубликаты разделенных номеров.

### Корректировка данных датафрейма топографического плана, при необходимости

Объединение разделенного номера в один при условии, что геометрия объекта равна у всех дубликатов.

In [5]:
topographic_plan = topographic_plan.drop_duplicates(subset=['split_number', 'geometry'])

topographic_plan

,origin_number,number_position,split_number,type,geometry,size
0,3,POINT (3108.838205298518 1707.285109871409),3,Point,POINT (3108.838205298518 1707.285109871409),NaN
1,4,POINT (3114.150988844972 1707.46955728739),4,Point,POINT (3114.150988844972 1707.46955728739),NaN
2,5,POINT (3115.690642463274 1707.58678205736),5,Point,POINT (3115.690642463274 1707.58678205736),NaN
3,6,POINT (3115.100445342571 1706.18062074892),6,Point,POINT (3115.100445342571 1706.18062074892),NaN
4,7,POINT (3114.699609216242 1705.090515570424),7,Point,POINT (3114.699609216242 1705.090515570424),NaN
5,10,POINT (3104.043722283615 1698.543651078497),10,Point,POINT (3104.043722283615 1698.543651078497),NaN
6,11,POINT (3105.090899215862 1695.948120853597),11,Point,POINT (3105.090899215862 1695.948120853597),NaN
7,12,POINT (3106.332887793542 1693.701322612177),12,Point,POINT (3106.332887793542 1693.701322612177),NaN
8,14,POINT (3104.361115387819 1692.888423763281),14,Point,POINT (3104.361115387819 1692.888423763281),NaN
9,13,POINT (3107.488870740065 1689.420932022136),13,Point,POINT (3107.488870740065 1689.420932022136),NaN


Разделение номеров, которые не прошли по заготовленным шаблонам регулярных выражений

In [6]:
# Единичные номера. Пример: ['71.2а', 'а5']
origin_numbers_1 = ['3']
for origin_number in origin_numbers_1:
    index = topographic_plan[topographic_plan['origin_number'] == origin_number].index[0]
    if topographic_plan.loc[index, 'origin_number'] is not None:
        print(f"Строка с origin_number={origin_number} содержит "
              f"split_number={topographic_plan.loc[index, 'split_number']}")
        continue
    topographic_plan.loc[index, 'split_number'] = origin_number
display(topographic_plan[topographic_plan['origin_number'].isin(origin_numbers_1)])

# Сборные номера. Пример [('27.1-27.3', ['27.1', '27.2', '27.3'])]
origin_split_numbers_2 = [
    ('27.1-27.3', ['27.1', '27.2', '27.3'])
]
for origin_number, split_numbers in origin_split_numbers_2:
    if len(topographic_plan[topographic_plan['origin_number'] == origin_number].index) > 1:
        print(f"Строка с origin_number={origin_number} возможно уже была обработана, так как таких строк несколько.")
        continue
    index = int(topographic_plan[topographic_plan['origin_number'] == origin_number].index[0])
    df_part1 = topographic_plan.iloc[:index]
    df_part2 = topographic_plan.iloc[index + 1:]
    additional_rows = []
    for split_number in split_numbers:
        duplicated_row = topographic_plan.loc[index].copy(deep=True)
        duplicated_row['split_number'] = split_number
        additional_rows.append(duplicated_row)
    topographic_plan = pd.concat([df_part1, pd.DataFrame(additional_rows), df_part2], ignore_index=True)
display(topographic_plan[topographic_plan['origin_number'].isin(
    [origin_numbers[0] for origin_numbers in origin_split_numbers_2]
)])

Строка с origin_number=3 содержит split_number=3


,origin_number,number_position,split_number,type,geometry,size
0,3,POINT (3108.838205298518 1707.285109871409),3,Point,POINT (3108.838205298518 1707.285109871409),NaN


Строка с origin_number=27.1-27.3 возможно уже была обработана, так как таких строк несколько.


,origin_number,number_position,split_number,type,geometry,size
23,27.1-27.3,POINT (3102.447619151131 1705.577447903106),27.1,Point,POINT (3102.447619151131 1705.577447903106),NaN
24,27.1-27.3,POINT (3102.447619151131 1705.577447903106),27.2,Point,POINT (3102.447619151131 1705.577447903106),NaN
25,27.1-27.3,POINT (3102.447619151131 1705.577447903106),27.3,Point,POINT (3102.447619151131 1705.577447903106),NaN


Повторная проверка

In [7]:
test_unsplit_number()

In [8]:
test_duplicated_slit_number()

### Вывод данных топографического плана

In [9]:
print('Статистика датафрейма')
print(f'\tВсего исходных номеров: {len(topographic_plan.origin_number.unique())}')
print(f'\tВсего разделенных номеров: {topographic_plan.split_number.count()}')
print(f'\tОтдельностоящих деревьев: {topographic_plan.type[topographic_plan.type == "Point"].count()}')
print(f'\tПолос деревьев: {topographic_plan.type[topographic_plan.type == "LineString"].count()}')
print(f'\tКонтуров растительности: {topographic_plan.type[topographic_plan.type == "Polygon"].count()}')

print('\nПОЛОСЫ ДЕРЕВЬЕВ, КУСТАРНИКОВ')
display(topographic_plan[topographic_plan.type == "LineString"])
print('\nКОНТУРЫ РАСТИТЕЛЬНОСТИ')
display(topographic_plan[topographic_plan.type == "Polygon"])
print('\nДЕРЕВЬЯ, КУСТАРНИКИ')
display(topographic_plan[topographic_plan.type == "Point"])

Статистика датафрейма
	Всего исходных номеров: 27
	Всего разделенных номеров: 34
	Отдельностоящих деревьев: 26
	Полос деревьев: 2
	Контуров растительности: 6

ПОЛОСЫ ДЕРЕВЬЕВ, КУСТАРНИКОВ


,origin_number,number_position,split_number,type,geometry,size
26,15,POINT (3092.268998306507 1693.40414713145),15,LineString,LINESTRING (3092.268998306507 1693.40414713145...,18.868199
27,16,POINT (3099.275403871401 1706.814426161843),16,LineString,LINESTRING (3099.275403871401 1706.81442616184...,6.236122



КОНТУРЫ РАСТИТЕЛЬНОСТИ


,origin_number,number_position,split_number,type,geometry,size
28,8,POINT (3106.665839634858 1703.063361392738),8,Polygon,"POLYGON ((3106.916523995033 1690.249262830753,...",72.187969
29,9,POINT (3104.129760581689 1690.619323186252),9,Polygon,"POLYGON ((3106.916523995033 1690.249262830753,...",72.187969
30,1,POINT (3075.965582309878 1696.605175781559),1,Polygon,"POLYGON ((3075.045526933311 1693.259229309259,...",13.829652
31,2,POINT (3074.823165523758 1693.90068742274),2,Polygon,"POLYGON ((3075.045526933311 1693.259229309259,...",13.829652
32,17,POINT (3103.397247052118 1686.424149899062),17,Polygon,"POLYGON ((3106.496601561342 1689.562929311654,...",13.404293
33,28,POINT (3097.319681942773 1694.253334704742),28,Polygon,"POLYGON ((3100.419036451997 1697.392114117334,...",13.404293



ДЕРЕВЬЯ, КУСТАРНИКИ


,origin_number,number_position,split_number,type,geometry,size
0,3,POINT (3108.838205298518 1707.285109871409),3,Point,POINT (3108.838205298518 1707.285109871409),NaN
1,4,POINT (3114.150988844972 1707.46955728739),4,Point,POINT (3114.150988844972 1707.46955728739),NaN
2,5,POINT (3115.690642463274 1707.58678205736),5,Point,POINT (3115.690642463274 1707.58678205736),NaN
3,6,POINT (3115.100445342571 1706.18062074892),6,Point,POINT (3115.100445342571 1706.18062074892),NaN
4,7,POINT (3114.699609216242 1705.090515570424),7,Point,POINT (3114.699609216242 1705.090515570424),NaN
5,10,POINT (3104.043722283615 1698.543651078497),10,Point,POINT (3104.043722283615 1698.543651078497),NaN
6,11,POINT (3105.090899215862 1695.948120853597),11,Point,POINT (3105.090899215862 1695.948120853597),NaN
7,12,POINT (3106.332887793542 1693.701322612177),12,Point,POINT (3106.332887793542 1693.701322612177),NaN
8,14,POINT (3104.361115387819 1692.888423763281),14,Point,POINT (3104.361115387819 1692.888423763281),NaN
9,13,POINT (3107.488870740065 1689.420932022136),13,Point,POINT (3107.488870740065 1689.420932022136),NaN


## Обработка ведомости таксации

Чтение docx или xls ведомости таксации и создание объекта ведомости таксации

In [22]:
taxation_list = create_taxation_list(
    file_path=Path("data/example_1_taxation_list.xlsx"),
    is_import_first_row=False,
    column_mapping={
        'number': 0,
        'name': 1,
        'quantity': 2,
        'height': 3,
        'diameter': 4,
        'quality': 5
    }
)
taxation_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   origin_number  26 non-null     object
 1   name           26 non-null     object
 2   quantity       26 non-null     object
 3   height         26 non-null     object
 4   diameter       26 non-null     object
 5   quality        26 non-null     object
dtypes: object(6)
memory usage: 1.3+ KB


Проверка ведомости таксации на дубликаты номеров

In [21]:
if taxation_list.duplicated(subset=['origin_number'], keep=False).sum() > 1:
    display(taxation_list[taxation_list.duplicated(subset=['origin_number'], keep=False)])
    raise ValueError(f"Обнаружены дубликаты номеров.")

,origin_number,name,quantity,height,diameter,quality
381,378,Ель,3,4х3,"0,05х3",Хорошее
382,379,Ель,1,6,"0,06",Хорошее
683,378,Орешник,5 стволов,4х5,"0,03х5",Хорошее
684,379,Ель,1,14,"0,20",Хорошее
2750,2725,Ольха,1,6,"0,05",Хорошее
5784,2725,Ива,1,8,"0,09",Хорошее
6224,6161,Ива,1,7,"0,05",Хорошее
6256,6161,Ива,3 ствола,7х3,"0,06; 0,08х2",Хорошее


ValueError: Обнаружены дубликаты номеров.